In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [48]:
from google.colab import files
uploaded = files.upload()


Saving mar.txt to mar (1).txt


In [0]:
import io
import pandas as pd
lines=pd.read_table(io.BytesIO(uploaded['mar.txt']), names=['eng', 'mar', 'source'])

In [50]:
lines['eng']

0                                                      Go.
1                                                     Run!
2                                                     Run!
3                                                     Run!
4                                                     Run!
                               ...                        
36498    If religion were synonymous with morality, Bra...
36499    Just saying you don't like fish because of the...
36500    Tom tried to sell his old VCR instead of throw...
36501    You can't view Flash content on an iPad. Howev...
36502    In 1969, Roger Miller recorded a song called "...
Name: eng, Length: 36503, dtype: object

In [0]:
# preprocessing

# lowercase 
lines.eng=lines.eng.apply(lambda x : x.lower())
lines.mar=lines.mar.apply(lambda x : x.lower())

# remove quotes
lines.eng=lines.eng.apply(lambda x : re.sub("'", "", x))
lines.mar=lines.mar.apply(lambda x : re.sub("'", "", x))

# removing special characters
exclude=set(string.punctuation)
lines.eng=lines.eng.apply(lambda x : ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x : ''.join(ch for ch in x if ch not in exclude))

# removing numbers
remove_digits=str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x : x.translate(remove_digits))
lines.mar=lines.mar.apply(lambda x : re.sub("[२३०८१५७९४६]", "", x))

# removing extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

# adding start and end tokens
lines.mar=lines.mar.apply(lambda x : '<start> '+x+' <end>')

In [52]:
lines.sample(10)

,eng,mar,source
18607,theres snow everywhere,<start> सगळीकडे बर्फ आहे <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
9703,we found her alive,<start> आम्हाला त्या जिवंत सापडल्या <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
8640,how can we do this,<start> आपण हे कसं करू शकतो <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
20495,tom will be back by,<start> टॉम अडीच वाजेपर्यंत परत येईल <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
29634,there are many islands in greece,<start> ग्रीसमध्ये पुष्कळ बेटं आहेत <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
14050,could it happen today,<start> आज घडू शकेल का <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
29061,tom wasnt doing anything wrong,<start> टॉम चुकीची गोष्ट करत नव्हता <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
25803,my uncle bought me this book,<start> माझ्या काकाने माझ्यासाठी हे पुस्तक विक...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
26636,i didnt know tom at the time,<start> त्यावेळी मी टॉमला ओळखत नव्हते <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
4120,i want to speak,<start> मला बोलायचं होतं <end>,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [53]:
lines=lines.iloc[:, 0:2]
lines

,eng,mar
0,go,<start> जा <end>
1,run,<start> पळ <end>
2,run,<start> धाव <end>
3,run,<start> पळा <end>
4,run,<start> धावा <end>
...,...,...
36498,if religion were synonymous with morality braz...,<start> जर धर्म व नीतिमत्ता समानार्थी शब्द असत...
36499,just saying you dont like fish because of the ...,<start> हड्डींमुळे मासे आवडत नाही असं म्हणणं ह...
36500,tom tried to sell his old vcr instead of throw...,<start> टॉमने त्याचा जुना व्हीसीआर फेकून टाकण्...
36501,you cant view flash content on an ipad however...,<start> आयपॅडवर फ्लॅश आशय बघता येत नाही पण तुम...


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
            
all_eng_words.add("pad")

# Vocabulary of Marathi 
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [9]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

34

In [10]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

37

In [11]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(5487, 13061)

In [0]:
num_decoder_tokens += 1 # For zero padding


In [0]:
num_encoder_tokens+=1

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [15]:
target_token_index, input_token_index

({'10': 1,
  '100': 2,
  '300': 3,
  '3ds': 4,
  '<end>': 5,
  '<start>': 6,
  'a': 7,
  'b': 8,
  'h₂o': 9,
  'ntt': 10,
  'tatoebaorg': 11,
  'uk': 12,
  'अ': 13,
  'अँकरेजमार्गे': 14,
  'अँजिलीस': 15,
  'अँड': 16,
  'अँडी': 17,
  'अँडीजमधल्या': 18,
  'अँब्युलन्स': 19,
  'अंक': 20,
  'अंकांनी': 21,
  'अंकारा': 22,
  'अंकारात': 23,
  'अंकारामध्ये': 24,
  'अंग': 25,
  'अंगठ्या': 26,
  'अंगाला': 27,
  'अंगोला': 28,
  'अंघोळ': 29,
  'अंठ्यांपेक्षा': 30,
  'अंड': 31,
  'अंडरवुडचा': 32,
  'अंडरवेअर': 33,
  'अंडी': 34,
  'अंडींचा': 35,
  'अंडीसुद्धा': 36,
  'अंड्यांवर': 37,
  'अंत': 38,
  'अंतर': 39,
  'अंतराळयात्री': 40,
  'अंतराळयान': 41,
  'अंतराळयानाचं': 42,
  'अंतराळयानातून': 43,
  'अंतराळवीर': 44,
  'अंतराळात': 45,
  'अंतरित': 46,
  'अंतिम': 47,
  'अंदाज': 48,
  'अंदाजे': 49,
  'अंधार': 50,
  'अंधाराची': 51,
  'अंधारात': 52,
  'अंशांकित': 53,
  'अकरा': 54,
  'अकरावीत': 55,
  'अकल्पनीय': 56,
  'अकाउंट': 57,
  'अकाउंटमध्ये': 58,
  'अकॉर्डियन': 59,
  'अक्कलदाढ': 60,
  'अक्वॅरियममध्ये': 6

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [0]:
lines = shuffle(lines)

In [18]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((32852,), (3651,))

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:
latent_dim=50

In [21]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [25]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)




Epoch 1/50





256/256 [==============================] - 89s 346ms/step - loss: 6.4723 - acc: 0.1776 - val_loss: 5.7963 - val_acc: 0.1839
Epoch 2/50
256/256 [==============================] - 78s 304ms/step - loss: 5.6021 - acc: 0.2079 - val_loss: 5.4627 - val_acc: 0.2217
Epoch 3/50
256/256 [==============================] - 77s 301ms/step - loss: 5.2936 - acc: 0.2345 - val_loss: 5.2400 - val_acc: 0.2445
Epoch 4/50
256/256 [==============================] - 76s 298ms/step - loss: 5.0654 - acc: 0.2609 - val_loss: 5.0538 - val_acc: 0.2660
Epoch 5/50
256/256 [==============================] - 76s 296ms/step - loss: 4.8643 - acc: 0.2784 - val_loss: 4.8941 - val_acc: 0.2812
Epoch 6/50
256/256 [==============================] - 76s 296ms/step - loss: 4.6852 - acc: 0.2968 - val_loss: 4.7547 - val_acc: 0.3008
Epoch 7/50
256/256 [==============================] - 75s 295ms/step - loss: 4.5284 - acc: 0.3138 - val_loss: 4.6291 - val_acc: 0.3157
Epoch 8/50
256/256 [===========================

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [32]:
!ls /content/gdrive/My Drive

ls: cannot access '/content/gdrive/My': No such file or directory
ls: cannot access 'Drive': No such file or directory


In [0]:
import torch

model_save_name = 'translator.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model, path)

In [0]:
model_save_name = 'translator.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
translator_model=torch.load(path)

In [0]:
# inference setup
encoder_model=Model(encoder_inputs, encoder_states)

# decoder setup
# previous state holders
decoder_state_input_h=Input(shape=(latent_dim, ))
decoder_state_input_c=Input(shape=(latent_dim, ))
decoder_state_inputs=[decoder_state_input_h, decoder_state_input_c]

dec_emb_2=dec_emb_layer(decoder_inputs)

decoder_outputs2, decoder_state_h2, decoder_state_c2=decoder_lstm(dec_emb_2, initial_state=decoder_state_inputs)
decoder_states_2=[decoder_state_h2, decoder_state_c2]
decoder_outputs2=decoder_dense(decoder_outputs2)

decoder_model=Model([decoder_inputs]+decoder_state_inputs, [decoder_outputs2]+decoder_states_2)

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['<start>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [65]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: they were silent
Actual Marathi Translation: > ते शांत होते <
Predicted Marathi Translation:  ते शांत होते <


In [66]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: ten is ten percent of one hundred
Actual Marathi Translation: > शंभरचे दहा टक्के म्हणजे दहा <
Predicted Marathi Translation:  दहा दहा बहीण आहेत हे मला माहीत आहे <


In [67]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom wrapped himself in his blanket
Actual Marathi Translation: > टॉमने स्वतःला आपल्या चादरीत गुंडाळून घेतलं <
Predicted Marathi Translation:  टॉमने आपल्या खिश्यातून प्रत्येक पत्र लागलं <
